In [2]:
from dataclasses import dataclass
!pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [7]:
import sqlite3

# Connect to the SQLite3 database in read-only mode
conn = sqlite3.connect('file:D:/.adds/koreader/settings/statistics.sqlite3?mode=ro', uri=True)

# Create a cursor object to interact with the database
cursor = conn.cursor()



In [8]:
cursor.execute('SELECT * FROM book')
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(2, 'The Shift Omnibus', 'Hugh Howey', 0, 1739700601, 0, 853, 'N/A', 'en', 'c3a8234c1bf4ee032bec78e42966a263', 2434, 34)


(1, 1, 1739639808, 9, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 2, 1739639822, 5, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 4, 1739639828, 16, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 6, 1739639847, 10, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 7, 1739640305, 53, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 6, 1739640298, 7, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(1, 6, 1739640358, 25, 8, 1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125

In [44]:
from datetime import datetime, timedelta
from dataclasses import dataclass

cursor.execute('SELECT * FROM page_stat_data psd LEFT JOIN book b ON b.id == psd.id_book')
rows = cursor.fetchall()

@dataclass
class Row:
    book_id: str
    # title: str
    # author: str
    page: int
    start_time: datetime
    duration: str
    time_difference: timedelta

converted_rows: [Row] = []

for row in rows:
    start_time_epoch = row[2]  # Assuming the start_time is in the second column
    duration_seconds = row[3]  # Assuming the duration is in the third column
    book_title = row[6]
    book_author = row[7]

    # Convert epoch time to human-readable format
    start_time = datetime.fromtimestamp(start_time_epoch)

    # Convert duration from seconds to a timedelta object
    duration = str(timedelta(seconds=duration_seconds))

    # converted_rows.append(Row(row[0], book_title, book_author, row[1], start_time, duration, None))
    converted_rows.append(Row(row[0], row[1], start_time, duration, None))

# Print the converted rows
converted_rows = sorted(converted_rows, key=lambda x: x.start_time)

previous_start_time = None

for row in converted_rows:
    start_time = row.start_time

    if row.duration == '0:02:00':
        continue

    if previous_start_time:
        #current_start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        row.time_difference = start_time - previous_start_time
        previous_start_time = start_time
    else:
        row.time_difference = 'N/A'
        previous_start_time = start_time # datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')




In [30]:

for row in converted_rows:
    if row.duration == '0:02:00':
        continue

    print(row.start_time, row.duration, row.time_difference)

# Close the connection

2025-02-15 17:16:48 0:00:09 N/A
2025-02-15 17:17:02 0:00:05 0:00:14
2025-02-15 17:17:08 0:00:16 0:00:06
2025-02-15 17:17:27 0:00:10 0:00:19
2025-02-15 17:24:58 0:00:07 0:07:31
2025-02-15 17:25:05 0:00:53 0:00:07
2025-02-15 17:25:58 0:00:25 0:00:53
2025-02-15 17:29:25 0:00:05 0:03:27
2025-02-15 17:29:32 0:00:37 0:00:07
2025-02-15 17:30:09 0:00:23 0:00:37
2025-02-15 17:30:37 0:00:59 0:00:28
2025-02-15 21:39:10 0:00:12 4:08:33
2025-02-15 21:41:50 0:00:06 0:02:40
2025-02-15 21:42:13 0:00:23 0:00:23
2025-02-15 21:42:37 0:00:14 0:00:24
2025-02-15 21:42:51 0:00:09 0:00:14
2025-02-15 22:21:23 0:00:16 0:38:32
2025-02-15 22:21:40 0:01:25 0:00:17
2025-02-15 22:23:05 0:01:20 0:01:25
2025-02-15 22:24:25 0:00:32 0:01:20
2025-02-15 22:24:57 0:00:32 0:00:32
2025-02-15 22:28:43 0:00:13 0:03:46
2025-02-15 22:40:59 0:01:00 0:12:16
2025-02-15 22:42:19 0:00:11 0:01:20
2025-02-15 22:42:33 0:00:05 0:00:14
2025-02-15 22:42:39 0:00:05 0:00:06
2025-02-15 22:42:44 0:01:09 0:00:05
2025-02-15 22:43:53 0:00:19 0:01

In [47]:
grouped_rows = {}

def get_day_and_night_of(start_time):
    if start_time.hour < 4:
        return (start_time - timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        return start_time.strftime('%Y-%m-%d')

for row in converted_rows:
    day_and_night_of = get_day_and_night_of(row.start_time)
    if day_and_night_of not in grouped_rows:
        grouped_rows[day_and_night_of] = []
    grouped_rows[day_and_night_of].append(row)


In [52]:
from sleep_events import connect_to_firebase

db = connect_to_firebase()

In [53]:
import json
from dataclasses import asdict
import sys

# Assuming grouped_rows is already populated
for day, rows in grouped_rows.items():
    json_rows = []
    for row in rows:
        if row.duration == '0:02:00':
            continue

        row_dict = asdict(row)
        row_dict['start_time'] = row.start_time.isoformat()
        row_dict['time_difference'] = str(row.time_difference)
        row_dict['day_and_night_of'] = day
        json_rows.append(row_dict)

    json_blob = json.dumps(json_rows, indent=4)
    print(f"JSON blob for {day}:")
    print(json_blob)

    # Get the size of json_blob in bytes
    json_blob_size = sys.getsizeof(json_blob)
    print(f"Size of json_blob for {day}: {json_blob_size} bytes")

    db.collection('kobo').document(day).set({'data': json_rows})

JSON blob for 2025-02-15:
[
    {
        "book_id": 1,
        "page": 1,
        "start_time": "2025-02-15T17:16:48",
        "duration": "0:00:09",
        "time_difference": "N/A",
        "day_and_night_of": "2025-02-15"
    },
    {
        "book_id": 1,
        "page": 2,
        "start_time": "2025-02-15T17:17:02",
        "duration": "0:00:05",
        "time_difference": "0:00:14",
        "day_and_night_of": "2025-02-15"
    },
    {
        "book_id": 1,
        "page": 4,
        "start_time": "2025-02-15T17:17:08",
        "duration": "0:00:16",
        "time_difference": "0:00:06",
        "day_and_night_of": "2025-02-15"
    },
    {
        "book_id": 1,
        "page": 6,
        "start_time": "2025-02-15T17:17:27",
        "duration": "0:00:10",
        "time_difference": "0:00:19",
        "day_and_night_of": "2025-02-15"
    },
    {
        "book_id": 1,
        "page": 6,
        "start_time": "2025-02-15T17:24:58",
        "duration": "0:00:07",
        "time_dif